#### create an env##
conda create -n macs2_env -c conda-forge -c bioconda -c anaconda macs2 python=3.8 r-irkernel ipykernel mamba

In [2]:
import subprocess
import glob
import os
from os.path import exists
import re
#from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool

In [3]:
output_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir'


In [4]:
cleaned_bam_dir = os.path.join(output_dir, '4_intersect_bam')
bam_files = glob.glob(os.path.join(cleaned_bam_dir, '*.bam'))
bam_files

['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12A_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM2_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM1_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/DM3_clean_filtered.bam',
 '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12B_clean_filtered.bam']

In [6]:
# Setting up directories for cleaned BAM files and not merged BAM file
peaks_dir = os.path.join(output_dir, '5_macs2peaks_simple')

os.makedirs(peaks_dir, exist_ok=True)


In [5]:
# Define the function to call peaks
def call_peaks(bam_file):
    output_prefix = os.path.join(peaks_dir, os.path.basename(bam_file).replace('.bam', ''))

    # Construct the MACS2 command
    command = [
        'macs2',
        'callpeak',
        '-t', bam_file,
        '-n', output_prefix,
        '--outdir', peaks_dir,
        '--format', 'BAMPE',
        '-g', '1.2e+8',  # genome size 120M for Ath genome
        '-B',  # This option tells MACS2 to save the pileup signal in a bedGraph file
    ]
    
    # Run the MACS2 command
    subprocess.run(command, check=True)

# Define the number of parallel processes
num_processes = 3 # Change this number as desired

# Use a Pool to process BAM files in parallel
with Pool(num_processes) as p:
    p.map(call_peaks, bam_files)


INFO  @ Tue, 31 Oct 2023 20:37:23: 
# Command line: callpeak -t /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam -n /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple/C12C_clean_filtered --outdir /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple --format BAMPE -g 1.2e+8 -B
# ARGUMENTS LIST:
# name = /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/5_macs2peaks_simple/C12C_clean_filtered
# format = BAMPE
# ChIP-seq file = ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam']
# control file = None
# effective genome size = 1.20e+08
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assign

In [11]:
# Setting up directories for cleaned BAM files and not merged BAM file
broad_peaks_dir = os.path.join(output_dir, '12_macs2peaks_broad2')

os.makedirs(broad_peaks_dir, exist_ok=True)


In [12]:
# Define the function to call peaks
def call_peaks_broad(bam_file):
    output_prefix = os.path.join(broad_peaks_dir, os.path.basename(bam_file).replace('.bam', ''))

    # Construct the MACS2 command
    command = [
        'macs2',
        'callpeak',
        '-t', bam_file,
        '-n', output_prefix,
        '--outdir', broad_peaks_dir,
        '--format', 'BAMPE',
        '-g', '1.2e+8',  # genome size 120M for Ath genome
        # '-B',  # This option tells MACS2 to save the pileup signal in a bedGraph file
        '--nomodel', 
        '--shift',  '-100',
        '--extsize', '200', 
        '--broad',
    ]
    
    # Run the MACS2 command
    subprocess.run(command, check=True)

# Define the number of parallel processes
num_processes = 3 # Change this number as desired

# Use a Pool to process BAM files in parallel
with Pool(num_processes) as p:
    p.map(call_peaks_broad, bam_files)


INFO  @ Wed, 24 Jan 2024 14:18:53: 
# Command line: callpeak -t /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam -n /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad2/C12C_clean_filtered --outdir /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad2 --format BAMPE -g 1.2e+8 -B --nomodel --shift -100 --extsize 200 --broad
# ARGUMENTS LIST:
# name = /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad2/C12C_clean_filtered
# format = BAMPE
# ChIP-seq file = ['/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/4_intersect_bam/C12C_clean_filtered.bam']
# control file = None
# effective genome size = 1.20e+08
# band width = 300
# model fold = [5, 50]
# qvalue cutoff for narrow/strong regions = 5.00e-02
# qvalue cutoff for broad/weak regions = 1.00e-01
